In [ ]:
%%javascript
require.config({
    paths: {
        flot: '//www.flotcharts.org/javascript/jquery.flot.min',
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

In [ ]:
%%html
<style>
div.output {
  max-height: 1000px;
  overflow-y: scroll;
}
</style>

In [ ]:
import numpy as np
import json
import time
from IPython.display import Javascript
from IPython.core.display import display

## Alpha

After running with the default alpha set below, try changing it to see if you can improve the convergence of the neural net.

In [ ]:
# Set alpha
# ----
# Alpha is the step size multiplier. Each time we adjust
# weights, we'll multiply the adjustment times alpha, 
# scaling the size of the adjustment during each update
alpha = 0.1

In [ ]:
# Activation function
# ----
# We'll use the sigmoid function as our activation function
def sigmoid(x):
    return 1/(1 + np.exp(-x))

# Activation derivative
# ----
# We use the derivative to find the direction
# of the gradient in SGD so we know which way (positive or negative)
# to move the weights when we update them
def sigmoid_output_derivative(output):
    return output*(1 - output)

# Inputs
X = np.array([[0,0,1],
             [0,1,1],
             [1,0,1],
             [1,1,1]])

# Outputs
Y = np.array([[0,1,1,0]]).T

# Seed a random number generator
np.random.seed(2)

# Randomly initialize weights between -1 and 1
W0 = 2*np.random.random((3,4)) - 1
W1 = 2*np.random.random((4,1)) - 1

# don't fret over all this nonsense,
# its just a bunch of javascript to set up the charts and layout
js = Javascript("""
    element.append("<h2 id='chart-header' style='text-align: center;'>Error, Weights, and Output Changing with Training Iterations</h2><br/><br />")
    element.append("<div id='chart' style='position: relative; width:90%; height: 400px; margin-top: -30px; margin-left: 20px;;'></div>")
    element.append("<div id='weight_charts' style='width: 100%; height: 320px; position: relative; padding-top:30px;'>" +
                     "<div id='weights_l0' style='position: absolute; width:27%; height: 250px; margin-left: 20px; padding-bottom: 30px;'></div>" +
                     "<div id='weights_l1' style='position: absolute; left:31%; width:27%; height: 250px; margin-left: 20px; padding-bottom: 30px;'></div>" +
                     "<div id='outputs' style='position: absolute; left:62%; width:27%; height: 250px; margin-left: 20px; padding-bottom: 30px;'></div>" +
                   "</div>")

    window.error = [];
    window.weights_l0 = [];
    window.weights_l1 = [];
    window.outputs = [];

    require(['flot'], function() {
      window.plot = $.plot('#chart', [[]], { series: { shadowSize: 0 }, yaxis: { min: 0, max: 0.5, transform: function(v) { return Math.log(v+1); }}, xaxis: { min: 0, max: 60000 } })
      var yaxisLabel = $("<div class='axisLabel yaxisLabel'></div>").text("Error").appendTo($('#chart'));
      yaxisLabel.css("position", "absolute").css("font-size","16px").css("margin-top", yaxisLabel.width() / 2 - 20);
      yaxisLabel.css("top", "50%").css("left", "-30px").css("transform","rotate(-90deg)");
      var xaxisLabel = $("<div class='axisLabel xaxisLabel'>Alpha: <span id='alpha'></span> &nbsp; Iteration: <span id='iteration'>0</span> &nbsp; Avg. Error: <span id='avg-error'></span></div></div>").appendTo($('#chart'));
      xaxisLabel.css("position", "absolute").css("width", "400px").css("font-size","16px").css("text-align","center");
      xaxisLabel.css("left", "-200px").css("bottom", "-20px").css("margin-left","50%");

      var w0_xaxisLabel = $("<div class='axisLabel xaxisLabel'></div>").text("Layer 0 Weights").appendTo($('#weight_charts'));
      w0_xaxisLabel.css("position", "absolute").css("font-size","16px").css("width","160px");
      w0_xaxisLabel.css("left", "-50px").css("bottom", "20px").css("margin-left", "16%");

      var w1_xaxisLabel = $("<div class='axisLabel xaxisLabel'></div>").text("Layer 1 Weights").appendTo($('#weight_charts'));
      w1_xaxisLabel.css("position", "absolute").css("font-size","16px").css("width","160px");
      w1_xaxisLabel.css("left", "-50px").css("bottom", "20px").css("margin-left", "47%");
      
      var outputs_xaxisLabel = $("<div class='axisLabel xaxisLabel'></div>").text("Outputs").appendTo($('#weight_charts'));
      outputs_xaxisLabel.css("position", "absolute").css("font-size","16px").css("width","160px");
      outputs_xaxisLabel.css("left", "-50px").css("bottom", "20px").css("margin-left", "81%");

      window.weight_plot_l0 = $.plot('#weights_l0', [[]], { series: { shadowSize: 0, bars: { show: true, align: "center", barWidth: 0.5 }}, yaxis: { min: -15, max: 15 }, xaxis: { min: 0, max: 13}})
      window.weight_plot_l1 = $.plot('#weights_l1', [[]], { series: { shadowSize: 0, bars: { show: true, align: "center", barWidth: 0.5 }}, yaxis: { min: -15, max: 15 }, xaxis: { min: 0, max: 5}})
      window.output_plot = $.plot('#outputs', [[]], { series: { shadowSize: 0, bars: { show: true, align: "center", barWidth: 0.5 }}, yaxis: { min: -0.5, max: 1.5 }, xaxis: { min: 0, max: 5}})
   
      $('#chart-header')[0].scrollIntoView();
   });
    
    
""")
display(js)

# give Javascript time to execute and render
time.sleep(1)

display(Javascript("$('#alpha').html(%.5f)" % alpha));

# Train
for i in range(1,60001):
    
    # Feed forward
    layer0 = X
    layer1 = sigmoid(np.dot(layer0,W0))
    layer2 = sigmoid(np.dot(layer1,W1))
    
    # Measure Layer 2 error
    layer2_error = layer2 - Y
    
    # Calculate Layer 2 update:
    # In what direction is the target value? Were we really sure?
    # If so, don't change too much
    # Multiply how much we missed by the slope of the sigmoid
    # at each value in Layer 2
    layer2_delta = layer2_error*sigmoid_output_derivative(layer2)
    
    # Measure Layer 1 error:
    # How much did each node in Layer 1 contribute to the 
    # Layer 2 error (according to the weights)
    layer1_error = np.dot(layer2_delta,W1.T)
    
    # Calculate Layer 1 update:
    # In which direction is the target Layer 1?
    layer1_delta = layer1_error*sigmoid_output_derivative(layer1)
    
    # Calculate weight update
    W1_update = alpha * np.dot(layer1.T, layer2_delta)
    W0_update = alpha * np.dot(layer0.T, layer1_delta)
    
    # Update weights
    W1 -= W1_update
    W0 -= W0_update
    
    if (i % 100 == 0):
        avg_error = np.mean(np.abs(layer2_error));
        weights_l0 = [[idx+1, weight] for idx, weight in enumerate(W0.flatten())]
        weights_l1 = [[idx+1, weight] for idx, weight in enumerate(W1.flatten())]
        outputs = [[idx+1, guess] for idx, guess in enumerate(layer2.flatten())]
        display(Javascript("$('#avg-error').html(%.5f)" % avg_error));
        display(Javascript("$('#iteration').html(%s)" % str(i)));
        display(Javascript("window.error.push([%d,%.5f])" % (i,avg_error)));
        display(Javascript("window.plot.setData([window.error]); window.plot.draw();"));
        display(Javascript("window.weights_l0 = %s" % json.dumps(weights_l0)));
        display(Javascript("window.weight_plot_l0.setData([window.weights_l0]); window.weight_plot_l0.draw();"));
        display(Javascript("window.weights_l1 = %s" % json.dumps(weights_l1)));
        display(Javascript("window.weight_plot_l1.setData([window.weights_l1]); window.weight_plot_l1.draw();"));
        display(Javascript("window.outputs = %s" % json.dumps(outputs)));
        display(Javascript("window.output_plot.setData([window.outputs]); window.output_plot.draw();"));
    